In [ ]:
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers.utils import load_image
from PIL import Image
import torch
import numpy as np
import cv2

def canny(img):
    image = np.array(img)
    image = cv2.Canny(image, 100, 200)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    image = Image.fromarray(image)
    
    return image

prompt = "Starry night beach scenery"
negative_prompt = 'low quality, bad quality, sketches'

image = load_image("data/test2.png")

controlnet_conditioning_scale = 0.7

controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-canny-sdxl-1.0",
    torch_dtype=torch.float16
)

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix",
                                    torch_dtype=torch.float16)

pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    vae=vae,
    torch_dtype=torch.float16,
)
pipe.enable_model_cpu_offload()

edge = canny(image)

images = pipe(
    prompt, negative_prompt=negative_prompt, image=edge, 
    controlnet_conditioning_scale=controlnet_conditioning_scale,
    ).images

images[0].save("controlnet_result.png")